# DIY Covid-19 Dashboard

The COVID 19 Pandemic is an ongoing global pandemic happening since 2019. Since december 2020 vaccines have been approved and widely distributed. This dashboard aims to analyse the impacts of the pandemic interms of the number of the number of cases and the number of fully vaccinated people, in which the affects of the vaccinations can be analysed.

In [1]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [2]:
%matplotlib inline
plt.rcParams['figure.dpi'] = 100

In [3]:

with open("covid_test_UK_vac.json","rt") as INFILE:
    data = json.load(INFILE)
jsondata={}

In [4]:
def parse_date(datestring):
    return pd.to_datetime(datestring, format="%Y-%m-%d")

def wrangle_data(data):
    data_list = data['data']
    date_dictionary= [dictionary['date'] for dictionary in data_list]
    date_dictionary.sort()
    start_date = parse_date(date_dictionary[0])
    end_date = parse_date(date_dictionary[-1])
    index=pd.date_range(start_date,end_date,freq='D')
    covid_uk_datadf = pd.DataFrame(index=index, columns =["fully_vac","new_cases"])
    for entry in data_list: 
        dataentry=entry['date']
        for column in ["fully_vac","new_cases"]:
            if pd.isna(covid_uk_datadf.loc[dataentry, column]): 
                value= float(entry[column]) if entry[column]!=None else 0.0# 
                covid_uk_datadf.loc[dataentry, column]=value
    return covid_uk_datadf

df=wrangle_data(data) 
df

,fully_vac,new_cases
2020-04-23,0.0,3589.0
2020-04-24,0.0,3565.0
2020-04-25,0.0,3472.0
2020-04-26,0.0,3218.0
2020-04-27,0.0,3155.0
...,...,...
2022-11-20,68.0,-45.0
2022-11-21,78.0,-45.0
2022-11-22,88.0,-45.0
2022-11-23,95.0,-45.0


In [5]:

def access_api():
    filters = ['areaType=overview']
    structure = {"date":"date",
             "fully_vac":"cumPeopleVaccinatedCompleteByVaccinationDate",
             "new_cases":"newCasesByPublishDate" }
    api = Cov19API(filters=filters,structure=structure)
    covid_test=api.get_json()
    apibutton.icon="check"
    apibutton.disabled=True
    return covid_test 

In [6]:

def api_button_callback(button):
   
    apidata=access_api()
    global df
    df=wrangle_data(apidata)
    apibutton.icon="check"

    
apibutton=wdg.Button(
    description='Refresh',
    disabled=False,
    button_style='info', 
    tooltip="Keep calm and order pineapple pizza",
    icon='info'
)


apibutton.on_click(api_button_callback) 

display(apibutton)



Button(button_style='info', description='Refresh', icon='info', style=ButtonStyle(), tooltip='Keep calm and or…

## Graphs and Analysis

This graph is a representation of the number of the number of cases and the number of fully vaccinated people, in which analysis on the impacts of the vaccine can be analysed, i.e to see how many people need vaccines.

In [7]:
def plot_random_walk(walk):
    df[walk].plot()
    plt.show() 
analysis_cols = wdg.SelectMultiple(
options =["fully_vac","new_cases"],
value=["fully_vac","new_cases"],
rows=2,
description="Fully Vac|Number of cases",
disabled=False)

scale =wdg.RadioButtons(
options = ['linear','log'],
descripion="Scale:",
disabled=False
)

controler = wdg.HBox([analysis_cols,scale])

def refresh_graph():
    current=analysis_cols.value
    if current==analysis_cols.options[0]:
        other=analysis_cols.options[1]
    else:
        other=analysis_cols.options[0]
    analysis_cols.value=other 
    analysis_cols.value=current


def analysis_graph(gcols,gscale):
    if gscale == 'linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        df[list(gcols)].plot(logy=logscale)
        plt.show()
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
        
    
    
graph=wdg.interactive_output(analysis_graph, {'gcols': analysis_cols, 'gscale': scale})

display(controler, graph)

Output()

**Author and Copyright Notice** Remember if you deploy this dashboard as a Binder it will be publicly accessible. Take credit for your work! Also acknowledge the data source: *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*